In [ ]:
!pip install -q torch transformers pandas numpy matplotlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the CSV data
file_path = './lr_dependency_results_scaled.csv'  # Update this path as necessary
data = pd.read_csv(file_path)

# Function to find minimums with random steps
def find_minimums_with_random_steps(data, steps):
    best_min = None
    best_value = np.inf
    mins = []
    indices_checked = set()

    for step in range(steps):
        while True:
            idx = np.random.randint(0, len(data))
            if idx not in indices_checked:
                indices_checked.add(idx)
                break

        # Check if the selected index is a minimum
        if (idx == 0 or data[idx] < data[idx - 1]) and (idx == len(data) - 1 or data[idx] < data[idx + 1]):
            if data[idx] < best_value:
                best_value = data[idx]
                best_min = idx
            mins.append((idx, data[idx], step))

    return mins, best_min, best_value

# Perform minimum finding and visualization for each question
questions = data['Question'].unique()
steps = 80

for question in questions:
    question_data = data[data['Question'] == question]
    loss_values = question_data['Loss'].values
    mins, min_index, min_value = find_minimums_with_random_steps(loss_values, steps)
    
    # Find the global best minimum
    global_best_min_index = np.argmin(loss_values)
    global_best_min_value = loss_values[global_best_min_index]

    # Print out the results
    print(f"Question: {question}")
    print(f"Global best minimum found at index {global_best_min_index} with value {global_best_min_value}")
    print(f"Best minimum found by the algorithm at index {min_index} with value {min_value}")
    print("\nMinimums found by the algorithm:")
    for idx, val, step in mins:
        print(f"Minimum at index {idx} with value {val} found at step {step}")

    # Plot the results
    plt.figure(figsize=(10, 6))
    plt.plot(question_data['Learning Rate'], loss_values, label='Loss')
    plt.plot(question_data['Learning Rate'].iloc[global_best_min_index], global_best_min_value, "o", label='Global Best Minimum', markersize=8)
    plt.plot(question_data['Learning Rate'].iloc[min_index], min_value, "x", label='Best Minimum Found', markersize=12)
    for p in mins:
        plt.plot(question_data['Learning Rate'].iloc[p[0]], p[1], "o", label='Minimum' if p == mins[0] else "")
    plt.yscale('log')  # Set y-axis to logarithmic scale
    plt.title(f'Minimum Detection for Question: {question}')
    plt.xlabel('Learning Rate')
    plt.ylabel('Loss (log scale)')
    plt.legend()
    plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the CSV data
file_path = 'lr_dependency_results_scaled.csv'  # Update this path as necessary
data = pd.read_csv(file_path)

# Function to find peaks with random steps and step search
def find_peaks_with_random_steps(data, steps):
    best_peak = None
    best_value = -np.inf
    peaks = []
    indices_checked = set()

    for step in range(steps):
        while True:
            idx = np.random.randint(0, len(data))
            if idx not in indices_checked:
                indices_checked.add(idx)
                break

        # Check left and right
        left_idx = idx
        while left_idx > 0 and data[left_idx - 1] < data[left_idx]:
            left_idx -= 1
            indices_checked.add(left_idx)

        right_idx = idx
        while right_idx < len(data) - 1 and data[right_idx + 1] < data[right_idx]:
            right_idx += 1
            indices_checked.add(right_idx)

        # Check if the selected index is a peak
        if (idx == 0 or data[idx] > data[idx - 1]) and (idx == len(data) - 1 or data[idx] > data[idx + 1]):
            if data[idx] > best_value:
                best_value = data[idx]
                best_peak = idx
            peaks.append((idx, data[idx], step))

    return peaks, best_peak, best_value

# Perform peak finding and visualization for each question
questions = data['Question'].unique()
steps = 80

for question in questions:
    question_data = data[data['Question'] == question]
    loss_values = question_data['Loss'].values
    peaks, peak_index, peak_value = find_peaks_with_random_steps(loss_values, steps)
    
    # Find the global best peak
    global_best_peak_index = np.argmax(loss_values)
    global_best_peak_value = loss_values[global_best_peak_index]

    # Print out the results
    print(f"Question: {question}")
    print(f"Global best peak found at index {global_best_peak_index} with value {global_best_peak_value}")
    print(f"Best peak found by the algorithm at index {peak_index} with value {peak_value}")
    print("\nPeaks found by the algorithm:")
    for idx, val, step in peaks:
        print(f"Peak at index {idx} with value {val} found at step {step}")

    # Plot the results
    plt.figure(figsize=(10, 6))
    plt.plot(question_data['Learning Rate'], loss_values, label='Loss')
    
    # Plot peaks with loss value <= 1
    if global_best_peak_value <= 1:
        plt.plot(question_data['Learning Rate'].iloc[global_best_peak_index], global_best_peak_value, "o", label='Global Best Peak', markersize=8)
    if peak_value <= 1:
        plt.plot(question_data['Learning Rate'].iloc[peak_index], peak_value, "x", label='Best Peak Found', markersize=12)
    for p in peaks:
        if p[1] <= 1:
            plt.plot(question_data['Learning Rate'].iloc[p[0]], p[1], "o", label='Peak' if p == peaks[0] else "")
    
    plt.title(f'Peak Detection for Question: {question}')
    plt.xlabel('Learning Rate')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
